# 1. Train a deeper Transformer in the experiments. How does it affect the training speed and the translation performance?

# 2. Is it a good idea to replace scaled dot product attention with additive attention in the Transformer? Why?

# 3. For language modeling, should we use the Transformer encoder, decoder, or both? How would you design this method?

# 4. What challenges can Transformers face if input sequences are very long? Why?

# 5. How would you improve the computational and memory efficiency of Transformers? Hint: you may refer to the survey paper by Tay et al. (2020).

In [1]:
import sys
import torch.nn as nn
import torch
import warnings
from sklearn.model_selection import ParameterGrid
sys.path.append('/home/jovyan/work/d2l_solutions/notebooks/exercises/d2l_utils/')
import d2l
from torchsummary import summary
warnings.filterwarnings("ignore")

class PositionWiseFFN(nn.Module):  #@save
    """The positionwise feed-forward network."""
    def __init__(self, ffn_num_hiddens, ffn_num_outputs):
        super().__init__()
        self.dense1 = nn.LazyLinear(ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.LazyLinear(ffn_num_outputs)

    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))

In [19]:
ffn = PositionWiseFFN(4, 8)
ffn.eval()
ffn(torch.ones((2, 3, 4)))[0]

tensor([[ 0.3717,  0.3617,  0.5549,  0.0816, -0.1657, -0.2510, -0.0093, -0.0123],
        [ 0.3717,  0.3617,  0.5549,  0.0816, -0.1657, -0.2510, -0.0093, -0.0123],
        [ 0.3717,  0.3617,  0.5549,  0.0816, -0.1657, -0.2510, -0.0093, -0.0123]],
       grad_fn=<SelectBackward0>)

In [21]:
ffn.dense1.weight.shape

torch.Size([4, 4])

In [25]:
ffn(torch.ones((2, 3, 4)))[0]

tensor([[ 0.3717,  0.3617,  0.5549,  0.0816, -0.1657, -0.2510, -0.0093, -0.0123],
        [ 0.3717,  0.3617,  0.5549,  0.0816, -0.1657, -0.2510, -0.0093, -0.0123],
        [ 0.3717,  0.3617,  0.5549,  0.0816, -0.1657, -0.2510, -0.0093, -0.0123]],
       grad_fn=<SelectBackward0>)

In [26]:
ffn(torch.ones((2, 1, 4)))[0]

tensor([[ 0.3717,  0.3617,  0.5549,  0.0816, -0.1657, -0.2510, -0.0093, -0.0123]],
       grad_fn=<SelectBackward0>)

In [28]:
ffn(torch.ones((2,3, 1, 4)))[0][0]

tensor([[ 0.3717,  0.3617,  0.5549,  0.0816, -0.1657, -0.2510, -0.0093, -0.0123]],
       grad_fn=<SelectBackward0>)